□ Are the statistical models appropriate given the data?  
✓ Did the author develop one or more machine learning models?  
✓ Did the author provide a way of assessing the performance and accuracy of their solution?  
□ Does the notebook have a **compelling and coherent narrative**?  

□ Does the notebook contain **data visualizations** that help to communicate the author’s main points?  
□ Did the author include a thorough discussion on the intersection between features and their prediction? For example between rainfall and amount/level of water.  
□ Was there discussion of **automated insight generation**, demonstrating what factors to take into account?  
✓ Is the code **documented** in a way that makes it easy to understand and reproduce?  
✓ Were all **external sources of data made public** and cited appropriately?  
✓ Is the provided model useful/able to forecast water availability in terms of level or water flow in a time interval of the year?  
✓ Is the provided methodology applicable also on new datasets belong to another waterbody?

---

# The Water Cycle

**Step 1**: Water falls out of the sky.

![Step 1 of the water cycle](images/cycle1.png)

**Step 2**: Water lands on the ground.

![Step 2 of the water cycle](images/cycle2.png)

Depending on where the water lands, it will either:

- 3a: Absorb into the soil
- 3b: Flow downhill

**Step 3a**: The ground soaks up the water.

Nearby plants will drink the water; otherwise, the water stays underground as an **aquifer**.

![Step 3a of the water cycle](images/cycle3a.png)

Sometimes, this groundwater will move through the land and come back out as a **water spring**.

![Step 3a2 of the water cycle](images/cycle3a2.png)

**Step 3b**: The ground *doesn't* soak up the water.

If the ground doesn't have little holes that water can soak into (eg, paved concrete, quartz, shale), or if the ground is already filled up with water, then the water will simply flow downhill and settle into the lowest part of the land.

![Step 3b of the water cycle](images/cycle3b.png)

A **river** eventually flows into an ocean. A **lake** might connect to a river, or it might just be an isolated body of water.

**Step 4**: While the water is sitting in a lake or flowing in a river, sunlight heats it up and causes some of it to float back into the sky.

![Step 4 of the water cycle](images/cycle4.png)

**Step 5**: And, of course, people can **remove** water from aquifers, springs, rivers, and lakes

![Step 5 of the water cycle](images/cycle5.png)

## How does this help us?

We can predict the amount of water in an **aquifer** if we know the following details:

- How much water fell from the sky?
- How **porous** and **permeable** is the ground it landed on?
- How much of that water was soaked up by vegetation?
- How empty was the aquifer when the water fell? In other words, how much **space** was available for groundwater recharge?
- How much water evaporated before it could soak into the ground?

We can predict the amount of water a **water spring** can give us if we know the following details:

- How much water is available from the groundwater? (Is the aquifer full? Empty?)
- How fast does the water flow underground?
- How much water is taken out before it can get to the spring?

And we can predict the amount of water in a **lake** or **river** if we know the following details:

- How much water fell out of the sky?
- How much water was lost to groundwater recharge or vegetation?
- How much water evaporated away?
- How much water was removed before we could measure it?
- How much water comes from other **tributaries**, other lakes, rivers, springs, and aquifers?

***

This notebook is designed first and foremost to make implementation and practical use of the models as quick as possible. Many technical points are hand-waved away behind convenience classes. But the overall effect of the code is explained as well, so that the techniques could be applied in any programming context.

## What are we building and why?

Acea serves water to over 9,000,000 inhabitants in Italy. They must be able to forecast the availability of water, from many different kinds of waterbodies, to ensure no one goes thirsty. We aim to develop four models designed to predict water availability from four types of waterbody:

- Aquifers
- Lakes
- Rivers
- Watersprings

Because each individual body of water has its own features and targets that indicate water availability, in all cases the models will not know in advance what features are available or what targets will be predicted.

We assume that the workflow at Acea is like so: Data not unlike the kind provided for this competition will be passed to the superior model, a specific interval of time will be chosen for the forecast (we compare accuracy at 5, 10, and 15 day intervals), and the values of the next step in time are predicted.

In [3]:
# dependencies
import pandas as pd
import numpy as np
import miceforest as mf
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

## Cleaning the Data

In [2]:
# please do not read this code it is scary and unenlightening
def drop_where_null(df, col):
    return df[~df[col].isnull()]

def date_as_index(df, col, format):
    df.index = pd.to_datetime(df[col], format=format)
    df.index.name = None
    df.index.freq = 'D'
    return df.drop(col, axis=1)

def start_after(df, y, m, d):
    return df[df.index > datetime(y, m, d)]

def trim_to_targets(df, y):
    notnulls = [df[~df[c].isnull()] for c in y]
    ends = [(nn.iloc[0].name, nn.iloc[-1].name) for nn in notnulls]
    firsts, lasts = zip(*ends)
    return df.loc[max(firsts):min(lasts)]

def reimpute_targets(df, y):
    for c in y:
        df[c] = df[c].replace(0, float('nan'))
    return df

def discard_sparse_features(df, x, thresh):
    todrop = []
    for c in x:
        ratio = df[c].isnull().sum() / df.shape[0]
        if ratio >= thresh:
            todrop.append(c)
    return df.drop(todrop, axis=1)

def absolute_columns(df, begs):
    cols = sum([[c for c in df.columns if c.startswith(b)] for b in begs], [])
    for c in cols:
        df[c] = df[c].abs()
    return df

def multiply_impute(df):               # USES RANDOMNESS
    kernel = mf.MultipleImputedKernel(
        data=df,
        save_all_iterations=False,
        random_state=143
    )
    kernel.mice(3, verbose=False)
    return kernel.impute_new_data(df).complete_data(0)

def aquifer_pipe(df, x, y):
    df = date_as_index(df, 'Date', '%d/%m/%Y')
    df = trim_to_targets(df, y)
    df = reimpute_targets(df, y)
    df = discard_sparse_features(df, x, .7)
    df = absolute_columns(df, ['Rainfall', 'Volume', 'Depth_to_Groundwater'])
    df = multiply_impute(df)
    return df

def waterspring_pipe(df, x, y):
    df = drop_where_null(df, 'Date')
    df = date_as_index(df, 'Date', '%d/%m/%Y')
    df = trim_to_targets(df, y)
    df = reimpute_targets(df, y)
    df = absolute_columns(df, ['Rainfall', 'Depth_to_Groundwater', 'Flow_Rate'])
    df = multiply_impute(df)
    return df

def river_pipe(df, x, y):
    df = drop_where_null(df, 'Date')
    df = date_as_index(df, 'Date', '%d/%m/%Y')
    df = trim_to_targets(df, y)
    df = reimpute_targets(df, y)
    df = discard_sparse_features(df, x, .7)
    df = absolute_columns(df, ['Rainfall'])
    df = multiply_impute(df)
    return df

def lake_pipe(df, x, y):
    df = date_as_index(df, 'Date', '%d/%m/%Y')
    df = start_after(df, 2004, 1, 1)
    df = absolute_columns(df, ['Rainfall', 'Flow_Rate', 'Lake_Level'])
    return df

We begin by creating four functions to clean, respectively, data from the four types of waterbodies. In all cases, these functions take three arguments: the complete, raw dataset as a Pandas dataframe, a list of the features, and a list of the targets.

Although the pipelines are indeed separate, the general flow goes like this:

- Drop where the date and targets are null
- Discard features with 70% nulls
- Use the absolute values of certain columns (as was specified in the dataset description file)
- Use multiple imputation to fill all remaining null values

As was mentioned above, we assume data used on a daily basis to make forecasts is similar to the data provided during the competition.

### Further Preprocessing

In [33]:
def chunk_days(df, ch):
    lose = df.shape[0] % ch
    df = df.iloc[lose:]
    newvals = {c:[] for c in df.columns}
    idx = []
    i = 0
    while i < df.shape[0]:
        vals = df.iloc[i:i+ch-1]
        idx.append(vals.iloc[0].name)
        for c in vals.columns:
            if c.startswith('Rainfall') or c.startswith('Volume'):
                newvals[c].append(vals[c].sum())
            else:
                newvals[c].append(vals[c].mean())
        i += ch
    return pd.DataFrame(newvals, index=idx, columns=df.columns)

class WaterbodyDataset:

    def __init__(self, cleaner, chunks, path, X, y):
        self.targets = y
        self.raw = pd.read_csv(path)
        self.clean = cleaner(self.raw, X, y)
        self.chunk = chunk_days(self.clean, chunks)
        scaler = StandardScaler()
        scaler.fit(self.chunk)
        self.stand = pd.DataFrame(
            scaler.transform(self.chunk),
            index=self.chunk.index,
            columns=self.chunk.columns
        )
        self.mu = pd.Series(scaler.mean_, index=self.chunk.columns)
        self.sigma = pd.Series(scaler.scale_, index=self.chunk.columns)

    def unscale(self, df):
        df = df[[c for c in df.columns if c in self.stand.columns]]
        for c in df.columns:
            df[c] = df[c] * self.sigma[c] + self.mu[c]
        return df

    def getX(self):
        return self.stand[list(set(self.stand.columns) - set(self.targets))]

    def gety(self):
        return self.stand[self.targets]

Two additional decisions were made in regards to processing data before training on it.

#### The Chunking Decision

We are investigating the accuracy of forecasts of lengths 5, 10, and 15 days. We assume that *the values of individual days are less important than the aggregated values over the full period*. When specifying a model, therefore, we must also tell it how many days to chunk into a single unit of time. (Rainfall and volume data are summed; all other features are aggregated with mean).

Our chunking function takes two arguments: the dataset and the number of days by which to chunk.

#### The Standardization Decision

The task at hand is inherently cross-units-of-measurement. We wish to see if a model performs better or worse on disparate types of data. In order to make these comparisons meaningful, we must first standardize our data and only then compute metrics.

However, we're trying to predict real values. So when values are plotted or shown to the reader, everything is unstandardized and framed in the original units of measurement. Accuracy metrics, as well, will be translated into the original units; just remember that we can only compare accuracy metrics across datasets if we use the ones based on standardized values.

Standardization, as well as other preprocessing discussed, is implemented in a class. The class is initialized with the function to be used for cleaning the data, the chunk interval, the path to the CSV containing our data, and the lists of features and targets. All cleaning and preprocessing is handled by this class, as well as exposing an `unscale` function that can take our standardized predictions and transform them back to the original scales.

## Models Under Investigation

In [4]:
# please do not read this code it is scary and unenlightening
class AlwaysZeroPredictor:
    name = 'zz'
    def fit(self, X, y):
        self.ycols = y
        return self
    def predict(self):
        return pd.DataFrame([[0] * self.outshape], columns=self.ycols)

class SimpleLinearPredictor:
    name = 'lm'
    def __init__(self):
        self.model = LinearRegression(fit_intercept=False, n_jobs=-1)
    def fit(self, X, y):
        self.ycols = y.columns
        self.last = X.iloc[-1]
        self.model.fit(X, y)
        return self
    def predict(self):
        return pd.DataFrame(self.model.predict([self.last]), columns=self.ycols)

Two predictive models were developed:

- Always predict 0, as a baseline (N.B.: because predictions are made using standardized scales, this translates into "always guess the mean")
- Linear regression + last-step-forward; in this case, we train a linear model on all available data, then use the most recent values in the training data to stand in for the predictors of the next step

## Validation & Accuracy

Validation of the data was performed with one-step-ahead forecasting. For each dataset provided, the first 67% of observations were used to train and forecast the rest of the data, one step per prediction.

Although RMSE and MAE were calculated, only RMSE is reported because the conclusions do not differ.

In [27]:
aquifer_rmse_scores = pd.DataFrame({
    'model':['Always predict 0/5 days','Always predict 0/10 days','Always predict 0/15 days',
             'Linear regression/5 days','Linear regression/10 days','Linear regression/15 days'],
    'auser':[0.8333082596275397,0.8330148075885803,0.8341275304853358,0.32761522075486665,0.3377017406276346,0.3760216798987572],
    'doganella':[1.0720980414361303,1.062223577480604,1.055159717184151,0.7650638972874075,0.7460756855996685,0.7216774611955543],
    'luco':[0.7560124618196891,0.7560833718330068,0.7508172684726705,0.6544776037245744,0.6268395754137174,0.6080520973079454],
    'petrignano':[0.4670167939762668,0.46768392090890987,0.46418502647850657,0.48071679020995045,0.4858255638613835,0.4750958430969966],
    'overall':[0.7821088892149065,0.7797514194527753,0.7760723856551659,0.5569683779941997,0.5491106413756011,0.5452117703748134]
}).sort_values('overall')
print('Aquifer RMSEs')
aquifer_rmse_scores

Aquifer RMSEs


,model,auser,doganella,luco,petrignano,overall
5,Linear regression/15 days,0.376022,0.721677,0.608052,0.475096,0.545212
4,Linear regression/10 days,0.337702,0.746076,0.626840,0.485826,0.549111
3,Linear regression/5 days,0.327615,0.765064,0.654478,0.480717,0.556968
2,Always predict 0/15 days,0.834128,1.055160,0.750817,0.464185,0.776072
1,Always predict 0/10 days,0.833015,1.062224,0.756083,0.467684,0.779751
0,Always predict 0/5 days,0.833308,1.072098,0.756012,0.467017,0.782109


In [28]:
lake_rmse_scores = pd.DataFrame({
    'model':['Always predict 0/5 days','Always predict 0/10 days','Always predict 0/15 days',
             'Linear regression/5 days','Linear regression/10 days','Linear regression/15 days'],
    'bilancino':[0.8354089525669881,0.8580082251171994,0.8629995370220535,0.8038668397322942,0.8098990983334172,0.8241646198642472]
}).sort_values('bilancino')
print('Lake RMSEs')
lake_rmse_scores

Lake RMSEs


,model,bilancino
3,Linear regression/5 days,0.803867
4,Linear regression/10 days,0.809899
5,Linear regression/15 days,0.824165
0,Always predict 0/5 days,0.835409
1,Always predict 0/10 days,0.858008
2,Always predict 0/15 days,0.863000


In [29]:
river_rmse_scores = pd.DataFrame({
    'model':['Always predict 0/5 days','Always predict 0/10 days','Always predict 0/15 days',
             'Linear regression/5 days','Linear regression/10 days','Linear regression/15 days'],
    'arno':[1.0919271183134502,1.0788395247525768,1.1079455437199186,0.8376263624372466,0.7723152452312124,0.7989117846124483]
}).sort_values('arno')
print('River RMSEs')
river_rmse_scores

River RMSEs


,model,arno
4,Linear regression/10 days,0.772315
5,Linear regression/15 days,0.798912
3,Linear regression/5 days,0.837626
1,Always predict 0/10 days,1.078840
0,Always predict 0/5 days,1.091927
2,Always predict 0/15 days,1.107946


In [30]:
waterspring_rmse_scores = pd.DataFrame({
    'model':['Always predict 0/5 days','Always predict 0/10 days','Always predict 0/15 days',
             'Linear regression/5 days','Linear regression/10 days','Linear regression/15 days'],
    'amiata':[0.7925895059639694,0.7923957134482332,0.7902828244301829,0.667815062180607,0.6634397909933601,0.6626858921926497],
    'lupa':[0.6813826699382083,0.6778626576038705,0.6962274993569028,0.6783021521550331,0.6707688514671745,0.6887722600350205],
    'madonna_di_canneto':[1.2398590926758895,1.2576003964265874,1.2671032532884259,1.2247627860621098,1.2400292880260622,1.2652356231244075],
    'overall':[0.9046104228593558,0.9092862558262303,0.9178711923585038,0.8569600001325833,0.858079310162199,0.8722312584506925]
}).sort_values('overall')
print('Waterspring RMSEs')
waterspring_rmse_scores

Waterspring RMSEs


,model,amiata,lupa,madonna_di_canneto,overall
3,Linear regression/5 days,0.667815,0.678302,1.224763,0.856960
4,Linear regression/10 days,0.663440,0.670769,1.240029,0.858079
5,Linear regression/15 days,0.662686,0.688772,1.265236,0.872231
0,Always predict 0/5 days,0.792590,0.681383,1.239859,0.904610
1,Always predict 0/10 days,0.792396,0.677863,1.257600,0.909286
2,Always predict 0/15 days,0.790283,0.696227,1.267103,0.917871


We see, then, that the best models for each type of waterbody are the following:

- **Aquifers**: Linear Regression, 15-day forecast interval
- **Lakes**: Linear Regression, 5-day forecast interval
- **Rivers**: Linear Regression, 10-day forecast interval
- **Watersprings**: Linear Regression, 5-day forecast interval

## How to Implement This Code

This notebook contains the following functions and classes:

- `aquifer_pipe`
- `lake_pipe`
- `river_pipe`
- `waterspring_pipe`
- `chunk_days`
- `WaterbodyDataset`
- `SimpleLinearPredictor`

If this code is simply copied/pasted, then training the model on some dataset is as simple as follows:

In [37]:
# we use the Auser aquifer data as an example

auser = WaterbodyDataset(
    cleaner=aquifer_pipe,
    chunks=15,
    path='data/raw/aquifer/auser.csv',
    X=[ 'Rainfall_Gallicano', 'Rainfall_Pontetetto', 'Rainfall_Monte_Serra', 'Rainfall_Orentano',
        'Rainfall_Borgo_a_Mozzano', 'Rainfall_Piaggione', 'Rainfall_Calavorno', 'Rainfall_Croce_Arcana',
        'Rainfall_Tereglio_Coreglia_Antelminelli', 'Rainfall_Fabbriche_di_Vallico', 'Depth_to_Groundwater_PAG',
        'Depth_to_Groundwater_DIEC', 'Temperature_Orentano', 'Temperature_Monte_Serra',
        'Temperature_Ponte_a_Moriano', 'Temperature_Lucca_Orto_Botanico', 'Volume_POL', 'Volume_CC1',
        'Volume_CC2', 'Volume_CSA', 'Volume_CSAL', 'Hydrometry_Monte_S_Quirico', 'Hydrometry_Piaggione' ],
    y=[ 'Depth_to_Groundwater_SAL', 'Depth_to_Groundwater_CoS', 'Depth_to_Groundwater_LT2' ]
)

prediction = SimpleLinearPredictor().fit(auser.getX(), auser.gety()).predict()
auser.unscale(prediction)

,Depth_to_Groundwater_SAL,Depth_to_Groundwater_CoS,Depth_to_Groundwater_LT2
0,5.241923,5.451136,12.397148


## Features & Their Usefulness

### Aquifer Auser

- `Rainfall_Gallicano` -- As far as I know, rainfall is helpful
- `Rainfall_Pontetetto`
- `Rainfall_Monte_Serra`
- `Rainfall_Orentano`
- `Rainfall_Borgo_a_Mozzano`
- `Rainfall_Piaggione`
- `Rainfall_Calavorno`
- `Rainfall_Croce_Arcana`
- `Rainfall_Tereglio_Coreglia_Antelminelli`
- `Rainfall_Fabbriche_di_Vallico`
- `Depth_to_Groundwater_PAG` -- As far as I know, depth to groundwater is helpful
- `Depth_to_Groundwater_DIEC`
- `Temperature_Orentano` -- As far as I know, temperature is helpful (but what if we don't have corresponding rainfall data?)
- `Temperature_Monte_Serra`
- `Temperature_Ponte_a_Moriano`
- `Temperature_Lucca_Orto_Botanico`
- `Volume_POL` -- ??? My guess is volume is not helpful
- `Volume_CC1`
- `Volume_CC2`
- `Volume_CSA`
- `Volume_CSAL`
- `Hydrometry_Monte_S_Quirico` -- Depending on how this differs from depth to groundwater, hydrometry is helpful
- `Hydrometry_Piaggione`

### Aquifer Doganella

- `Rainfall_Monteporzio`
- `Rainfall_Velletri`
- `Volume_Pozzo_1`
- `Volume_Pozzo_2`
- `Volume_Pozzo_3`
- `Volume_Pozzo_4`
- `Volume_Pozzo_5+6`
- `Volume_Pozzo_7`
- `Volume_Pozzo_8`
- `Volume_Pozzo_9`
- `Temperature_Monteporzio`
- `Temperature_Velletri`

Rainfall + temperature, AFAIK, are helpful.

Volume refers to amount of water taken at a treament facility; unless this amount is dictated by how much is available, how can this help?

### Aquifer Luco

- `Rainfall_Simignano`
- `Rainfall_Siena_Poggio_al_Vento`
- `Rainfall_Mensano`
- `Rainfall_Montalcinello`
- `Rainfall_Monticiano_la_Pineta`
- `Rainfall_Sovicille`
- `Rainfall_Ponte_Orgia`
- `Rainfall_Scorgiano`
- `Rainfall_Pentolina`
- `Rainfall_Monteroni_Arbia_Biena`
- `Depth_to_Groundwater_Pozzo_1`
- `Depth_to_Groundwater_Pozzo_3`
- `Depth_to_Groundwater_Pozzo_4`
- `Temperature_Siena_Poggio_al_Vento`
- `Temperature_Mensano`
- `Temperature_Pentolina`
- `Temperature_Monteroni_Arbia_Biena`
- `Volume_Pozzo_1`
- `Volume_Pozzo_3`
- `Volume_Pozzo_4`

See notes on other aquifers

### Aquifer Petrignano

- `Rainfall_Bastia_Umbra`
- `Temperature_Bastia_Umbra`
- `Temperature_Petrignano`
- `Volume_C10_Petrignano`
- `Hydrometry_Fiume_Chiascio_Petrignano` -- Dataset description file has this as groundwater level (ie, for an aquifer); but "fiume" is Italian for "river", so is it actually stage? If it's actually river level, surely this isn't helpful?

See notes for other aquifers

### Water Spring Amiata

- `Rainfall_Castel_del_Piano`
- `Rainfall_Abbadia_S_Salvatore`
- `Rainfall_S_Fiora`
- `Rainfall_Laghetto_Verde`
- `Rainfall_Vetta_Amiata`
- `Depth_to_Groundwater_S_Fiora_8`
- `Depth_to_Groundwater_S_Fiora_11bis`
- `Depth_to_Groundwater_David_Lazzaretti`
- `Temperature_Abbadia_S_Salvatore`
- `Temperature_S_Fiora`
- `Temperature_Laghetto_Verde`

As far as I know (and before I've plotted them all), these are all helpful features.

"Therefore, the groundwater level in the study area is more sensitive to temperature than to rainfall." --[Impact of temperature changes on groundwater levels... in Bangladesh](https://www.jstage.jst.go.jp/article/hrl/11/1/11_85/_pdf/-char/en)

### Water Spring Lupa

- `Rainfall_Terni`

Again, as far as I know, this is a useful feature; but we could recommend *more* useful feature types (Temp, Depth to Groundwater) depending on our research

### Water Spring Madonna di Canneto

- `Rainfall_Settefrati`
- `Temperature_Settefrati`

Both helpful (depending on location relative to where flow rate is measured)

### Lake Bilancino

- `Rainfall_S_Piero` **Not helpful.** Located on the river "after" the lake
- `Rainfall_Mangona` **Helpful, but** Located far (5+km) from the lake
- `Rainfall_S_Agata` **Helpful.** Located <5km from lake
- `Rainfall_Cavallina` **Helpful.** Located right on the lake
- `Rainfall_Le_Croci` **Helpful.** Located right next to the lake
- `Temperature_Le_Croci` **Helpful.** Located right next to the lake

### Arno River

- `Rainfall_Le_Croci` **Not helpful.** Separated from Arno by another system of water (Lake Bilancino)
- `Rainfall_Cavallina` **Not helpful.** Separated from Arno by another system of water (Lake Bilancino)
- `Rainfall_S_Agata` **Not helpful.** Separated from Arno by another system of water (Lake Bilancino), and some distance away from that one as well
- `Rainfall_Mangona` **Not helpful.** Separated from Arno by another system of water (Lake Bilancino), and some distance away from that one as well
- `Rainfall_S_Piero` **Not helpful.** Separated from Arno by another system of water (River Sieve)
- `Rainfall_Vernio` **Not helpful.** Located on a river (Bisenzio) that feeds into Arno *after* location where stage is measured
- `Rainfall_Stia` **Helpful.** Located right on the Arno
- `Rainfall_Consuma` **Not helpful.** Located too far away from Arno
- `Rainfall_Incisa` **Helpful.** Located right on the Arno
- `Rainfall_Montevarchi` **Helpful.** Located right on the Arno
- `Rainfall_S_Savino` **Not helpful.** Located too far away from Arno
- `Rainfall_Laterina` **Helpful.** Located right on the Arno
- `Rainfall_Bibbiena` **Helpful.** Located very close to the Arno
- `Rainfall_Camaldoli` **Not helpful.** Located too far away from Arno
- `Temperature_Firenze` **Not helpful.** Located too far away from where stage is measured for temperature to be useful